In [1]:
import os
from getpass import getpass

# Set up OpenAI API key
groq_api_key = getpass("Enter your Groq key: ")
os.environ["GROQ_API_KEY"] = groq_api_key

Enter your Groq key: ··········


In [2]:
!pip install -q langchain openai chromadb tiktoken pypdf langchain_openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.9/19.9 MB 71.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.5/323.5 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 61.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 75.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.8/510.8 kB 30.5 MB/s eta

In [3]:
!pip install -q langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [4]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

In [5]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

In [6]:
!pip install langchain-groq langchain duckduckgo-search langchain_community ddgs  --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 68.4 MB/s eta 0:00:00


In [55]:
from langchain_groq import ChatGroq
from langchain.memory import ConversationBufferWindowMemory

llm = ChatGroq(model="llama-3.1-8b-instant", temperature=0.5, max_tokens=1024)
memory = ConversationBufferWindowMemory(memory_key="chat_history", return_messages=True, k=5)

In [8]:
loader = PyPDFLoader("/content/Guidance-Document-on-TB-Mukt-Bharat-Abhiyan_0.pdf")
docs = loader.load()
splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=100)
chunks = splitter.split_documents(docs)

In [9]:
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"  # lightweight and fast
)

/tmp/ipython-input-2627326643.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
pip install langchain faiss-cpu sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 24.7 MB/s eta 0:00:00


In [11]:
vectorstore = FAISS.from_documents(chunks, embedding_model)
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})

In [59]:
from langchain.prompts import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,
)

system_template = """
You are TB-AwareGPT, an assistant for NGO volunteers.
Answer ONLY using the information in the provided source document chunks.
When volunteers mentions to respond Hindi, produce answers in Hindi.

Keep replies short and actionable.
- Always cite the source chunk id (metadata 'source').
- If the answer is not in the document or is not related to TB, say: "I cannot find that in the manual; please refer to the local PHC or call 1800-11-6666."
- Do not make up or assume anything outside the document.

Language control:
- If lang="hi", respond in Hindi.
- If lang="en", respond in English.
- Default to English unless explicitly told otherwise.

Examples (do NOT invent answers):
Q: Who is Sachin Tendulkar?
A: I cannot find that in the manual; please refer to the local PHC or call 1800-11-6666.

Q: What are TB symptoms?
A: The main symptoms are: persistent cough for 2 weeks or more, fever, night sweats, weight loss, fatigue. Source: chunk_12

Q: Are fever, night sweats, loss of appetite symptoms of TB?
A: Yes, fever, night sweats, loss of appetite are common symptons of TB. Source: chunk_12
"""

human_template = """
Use the following document snippets to answer the question.
If snippets do not contain the answer, reply: "I cannot find that in the manual; please refer to the local PHC or call 1800-11-6666."

--- Retrieved snippets:
{context}

--- Question:
{question}
"""

system_msg = SystemMessagePromptTemplate.from_template(system_template)
human_msg = HumanMessagePromptTemplate.from_template(human_template)
chat_prompt = ChatPromptTemplate.from_messages([system_msg, human_msg])

In [60]:
from langchain.chains import RetrievalQA
from langchain.schema import HumanMessage

system_instructions = """
You are TB-AwareGPT, an assistant for NGO volunteers.
Answer using ONLY the provided manual excerpts. Do NOT provide medical advice beyond the manual.
If the question is not related to TB, please say "Question not related to TB, please ask question related to TB"
When volunteers prefer Hindi, produce answers in Hindi. Keep replies short and actionable.
"""

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": chat_prompt},
)

In [61]:
def answer_query(user_name, question, lang="en"):
    # get retrieved docs and answer
    result = qa_chain({"query": question})
    answer = result["result"]  # the LLM-produced answer
    # Localization
    if lang.lower().startswith("hi"):
        # ask LLM to translate/compose in Hindi, but still constrained
        prompt = f"Translate and adapt to conversational Hindi the following factual answer (do not add new info):\n\n{answer}"
        return llm([HumanMessage(content=prompt)]).content
    return answer

In [62]:
def generate_quiz_from_topic(topic_query, n_questions=5, lang="en"):
    # Retrieve relevant chunks using the topic query
    relevant_docs = retriever.get_relevant_documents(topic_query)

    # Combine the text from top chunks
    section_text = "\n\n".join([doc.page_content for doc in relevant_docs])

    # Generate quiz from the combined section
    return generate_quiz(section_text, n_questions=n_questions, lang=lang)

def generate_quiz(section_text, n_questions=5, lang="en"):
    # Few-shot examples to guide format and tone
    quiz_few_shots = """
    Example:
    Q: What is the minimum duration of TB treatment?
    Options: ["2 weeks", "6 months", "1 year"]
    Answer: 1
    Explanation: TB treatment typically lasts at least 6 months.

    Q: Which symptom is most common in pulmonary TB?
    Options: ["Skin rash", "Persistent cough", "Joint pain"]
    Answer: 1
    Explanation: Persistent cough is a hallmark of pulmonary TB.
    """

    # Construct prompt with examples + section text
    prompt = f"""
    {quiz_few_shots}

    Now create {n_questions} quiz items based ONLY on the following text:
    {section_text}

    Output as JSON list: [{{"q": "...", "opts": ["...", "...", "..."], "a": 0, "ex": "..."}}]
    Respond in {'Hindi' if lang.lower().startswith('hi') else 'English'}.
    """
    return llm([HumanMessage(content=prompt)]).content

def grade_answer(quiz_item, given_answer_index):
    correct = quiz_item["a"]
    if given_answer_index == correct:
        return True, "Correct: " + quiz_item["ex"]
    return False, f"Incorrect. Correct: option {correct+1}. {quiz_item['ex']}"

In [63]:
query = "What are the common symptoms of TB?"
result = qa_chain.invoke({"query": query})
print("Answer:", result["result"])

Answer: The main symptoms are: persistent cough for 2 weeks or more, fever, chest pain, blood in sputum, night sweats, loss of appetite, weight loss, weakness or fatigue. Source: chunk_1


In [64]:
query = "what are symptoms of covid 19"
result = qa_chain.invoke({"query": query})
print("Answer:", result["result"])

Answer: I cannot find that in the manual; please refer to the local PHC or call 1800-11-6666.


In [65]:
query = "What are the 5 common symptoms of TB I should tell people about? Please answer in Hindi"
result = qa_chain.invoke({"query": query})
print("Answer:", result["result"])

Answer: कॉमन लक्षण हैं: 
- देर से दिन में खांसी, 
- बुखार, 
- सीने में दर्द, 
- खूनी सिरपेस, 
- रात में पसीना आना। 
स्रोत: chunk_1


In [66]:
query = "Are fever, chest pain , loss of appetite symptoms of TB?"
result = qa_chain.invoke({"query": query})
print("Answer:", result["result"])

Answer: Yes, fever, chest pain, and loss of appetite are common symptoms of TB. Source: chunk_1


In [67]:
response = answer_query("Sumit", "Are fever, night sweats, loss of appetite symptoms of TB?", lang="en")
print("Answer:", response)

Answer: Yes, fever, night sweats, loss of appetite are common symptoms of TB. Source: chunk_1


In [46]:
quiz = generate_quiz_from_topic("TB symptoms", n_questions=5, lang="en")
print(quiz)

Here are 5 quiz items based on the provided text:

[{"q": "What are the common symptoms of TB?", "opts": ["Cough, fever, chest pain", "Loss of appetite, weight loss, weakness", "Skin rash, joint pain, backache"], "a": 0, "ex": "Cough, fever, chest pain, blood in sputum, night sweats, loss of appetite, weight loss, weakness or fatigue."}]

[{"q": "How long should cough last to be considered a symptom of TB in vulnerable populations?", "opts": ["1 week", "2 weeks", "1 month"], "a": 1, "ex": "Cough >2 weeks (any duration if vulnerable population)"},]

[{"q": "What is the recommended action for health workers to identify TB?", "opts": ["Use X-rays for screening", "Line list vulnerable individuals and screen with 10-S tool", "Refer to Annexure 1: Algorithm for TB screening"], "a": 1, "ex": "Health workers should be trained to line list the vulnerable and screen them with 10-S tool,"}]

[{"q": "Who are considered high-risk groups for TB?", "opts": ["People living in urban areas", "Individual

In [68]:
quiz = generate_quiz_from_topic("TB symptoms", n_questions=5, lang="hi")
print(quiz)

निम्नलिखित 5 प्रश्नों के उत्तर दिए गए हैं:

```json
[
  {"q": "TB के लक्षणों में से कौन सा सबसे आम है?", "opts": ["त्वचा का चकत्ता", "अनवरत खांसी", "जोड़ों का दर्द"], "a": 1, "ex": "अनवरत खांसी TB का एक आम लक्षण है."},

  {"q": "TB के लक्षणों में से कौन सा सबसे आम है जब बच्चे हों?", "opts": ["बुखार", "खांसी", "वजन कमी"], "a": 2, "ex": "बच्चों में वजन कमी एक आम लक्षण है."},

  {"q": "TB के लक्षणों में से कौन सा सबसे आम है जब वयस्क हों?", "opts": ["बुखार", "खांसी", "वजन कमी"], "a": 1, "ex": "वयस्कों में बुखार एक आम लक्षण है."},

  {"q": "TB से बचाव के लिए क्या करना चाहिए?", "opts": ["अनवरत खांसी", "बुखार", "साँस लेने का सही तरीका"], "a": 2, "ex": "साँस लेने का सही तरीका और उचित वेंटिलेशन से TB से बचाव होता है."},

  {"q": "TB के लक्षणों का पता लगाने के लिए क्या करना चाहिए?", "opts": ["सिर्फ खांसी और बुखार की जांच करना", "अन्य लक्षणों की भी जांच करना", "केवल वयस्कों की जांच करना"], "a": 1, "ex": "अन्य लक्षणों की भी जांच करना चाहिए, जैसे कि सूजन, जोड़ों का दर्द, पेट दर्द आदि."}
]
```

यह प

In [74]:
intent_examples = {
    "generate_quiz": [
        "Give me a quiz on TB symptoms",
        "Create MCQs for TB awareness",
        "Can I get some practice questions?"
    ],
    "answer_query": [
        "How can we prevent TB?",
        "What are TB symptoms?",
        "Tell me about TB treatment duration"
    ],
    "grade_answer": [
        "Evaluate my quiz answers",
        "Did I get this question right?",
        "Score my response"
    ]
}

In [75]:
from langchain.schema import Document

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Flatten examples into documents with metadata
intent_docs = []
for intent, examples in intent_examples.items():
    for ex in examples:
        intent_docs.append(Document(page_content=ex, metadata={"intent": intent}))

intent_index = FAISS.from_documents(intent_docs, embedding_model)

In [76]:
def route_intent_semantic(user_input):
    results = intent_index.similarity_search(user_input, k=1)
    if results:
        return results[0].metadata["intent"]
    return "default"

In [77]:
user_input = "Can you give me a quiz on TB symptoms?"
intent = route_intent_semantic(user_input)

def ask_llm(intent):
  if intent == "generate_quiz":
      response = generate_quiz_from_topic(user_input, n_questions=5, lang="en")
  elif intent == "answer_query":
      response = answer_query("Sumit", user_input, lang="en")
  elif intent == "grade_answer":
      response = "Grading not yet wired—coming soon!"
  else:
      response = "Sorry, I couldn't understand your request. Please try again."
  return response



In [78]:
user_input = "Can you give me some TB symptoms?"
intent = route_intent_semantic(user_input)
response = ask_llm(intent)
print("Response:", response)

Response: The main symptoms are: 
- Persistent cough for 2 weeks or more
- Fever
- Chest pain
- Blood in sputum
- Night sweats
- Loss of appetite
- Weight loss
- Fatigue

Source: chunk_1


In [80]:
user_input = "Can you give some basic ways we can prevent TB from spreading? Can you please reply in Hindi"
intent = route_intent_semantic(user_input)
response = ask_llm(intent)
print("Response:", response)

Response: TB से फैलाव रोकने के कुछ मूलभूत तरीके हैं:

1. सांस की बीमारी की जानकारी फैलाएं।
2. सामूहिक स्थानों में सांस की बीमारी के प्रति जागरूकता बढ़ाएं।
3. स्वस्थ आहार लें और स्वस्थ जीवनशैली अपनाएं।
4. सामाजिक समर्थन प्राप्त करें। (स्रोत: chunk_5)


In [81]:
user_input = "Can you give some basic ways we can prevent TB from spreading?"
intent = route_intent_semantic(user_input)
response = ask_llm(intent)
print("Response:", response)

Response: Maintain cough hygiene, nutrition, avoid overcrowding, adequate ventilation, TB preventive treatment and healthy behavior. Source: chunk_5
